In [ ]:
import pandas as pd
import os
import pandas as pd
#!pip install PyPDF2
from PyPDF2 import PdfReader
from transformers import pipeline#

In [ ]:
# function to extract text from PDF files
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text.strip()

# DataFrame to store the extracted data
columns = ["document","category"]
data = []

# Iterate over the PDF files and extract text
dataset_dir = r"/content/drive/MyDrive/documnet_clasification/dataset/dataset_pdf"
for category in os.listdir(dataset_dir):
    category_dir = os.path.join(dataset_dir, category)
    if os.path.isdir(category_dir):
        for file_name in os.listdir(category_dir):
            if file_name.endswith(".pdf"):
                pdf_path = os.path.join(category_dir, file_name)
                document = extract_text_from_pdf(pdf_path)
                data.append([document,category])

# Create DataFrame
df = pd.DataFrame(data, columns=columns)

print(df.head())

In [11]:
#!pip install datasets
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import DatasetDict, Dataset

#df = pd.read_csv("/content/drive/MyDrive/bbc-text.csv")[["text","category"]]
df = df.rename(columns={ 'document':'text','category': 'label_text'})


label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label_text'])



df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)



# Assuming 'df_train' and 'df_test' are your DataFrames for training and testing data respectively
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)


train_dataset = train_dataset.remove_columns('__index_level_0__')
test_dataset = test_dataset.remove_columns('__index_level_0__')
# Define the features of the datasets
features = {
    'document': ('string',),
    'category': ('string',)
}

# Set the features of the datasets
#train_dataset = train_dataset.cast(features)
#test_dataset = test_dataset.cast(features)

# Create DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label_text', 'label'],
        num_rows: 1557
    })
    test: Dataset({
        features: ['text', 'label_text', 'label'],
        num_rows: 668
    })
})

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label_text', 'label'],
        num_rows: 1557
    })
    test: Dataset({
        features: ['text', 'label_text', 'label'],
        num_rows: 668
    })
})

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 3534
    })
})

In [13]:
from datasets import load_dataset
import pandas as pd
from transformers import GPT2Tokenizer

#dataset = load_dataset("mteb/tweet_sentiment_extraction")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

In [14]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(400))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(400))

In [15]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=5)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
#!pip install evaluate
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [19]:
#!pip install transformers[torch]
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
   output_dir="/content/drive/MyDrive/test_trainer",
   #evaluation_strategy="epoch",
   per_device_train_batch_size=1,  # Reduce batch size here
   per_device_eval_batch_size=1,    # Optionally, reduce for evaluation as well
   gradient_accumulation_steps=4
   )


trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,

)

trainer.train()


Step,Training Loss


TrainOutput(global_step=300, training_loss=0.14647216796875, metrics={'train_runtime': 327.5362, 'train_samples_per_second': 3.664, 'train_steps_per_second': 0.916, 'total_flos': 627129188352000.0, 'train_loss': 0.14647216796875, 'epoch': 3.0})

In [22]:
import evaluate

trainer.evaluate()


{'eval_loss': 0.15753690898418427,
 'eval_accuracy': 0.97,
 'eval_runtime': 38.1782,
 'eval_samples_per_second': 10.477,
 'eval_steps_per_second': 10.477,
 'epoch': 3.0}

In [25]:
model_path = "/content/drive/MyDrive/saved_model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/saved_model/tokenizer_config.json',
 '/content/drive/MyDrive/saved_model/special_tokens_map.json',
 '/content/drive/MyDrive/saved_model/vocab.json',
 '/content/drive/MyDrive/saved_model/merges.txt',
 '/content/drive/MyDrive/saved_model/added_tokens.json')

In [31]:
df.head(5)

,text,label_text,label
0,tv future in the hands of viewers with home th...,tech,4
1,worldcom boss left books alone former worldc...,business,0
2,tigers wary of farrell gamble leicester say ...,sport,3
3,yeading face newcastle in fa cup premiership s...,sport,3
4,ocean s twelve raids box office ocean s twelve...,entertainment,1


In [43]:
#New Prediction

from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
import torch
#!pip install PyPDF2
from PyPDF2 import PdfReader

# Load the saved model and tokenizer
model = GPT2ForSequenceClassification.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)


def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text.strip()

# Define your new text data
pdf_path = "/content/drive/MyDrive/text_7.pdf"
new_text = extract_text_from_pdf(pdf_path)
#new_text =df["text"][0]

# Tokenize and preprocess the new text data
inputs = tokenizer(new_text, padding="max_length", truncation=True, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted label
predicted_label = torch.argmax(outputs.logits).item()

# Decode the predicted label
label_text = label_encoder.inverse_transform([predicted_label])[0]

print("Predicted Label:", label_text)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.8 MB/s eta 0:00:00
Predicted Label: politics
